In [1]:
import pickle
from keras.models import load_model
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from numpy import asarray
from numpy import save

with open('training.pkl', 'rb') as f:
     train = pickle.load(f)
    
with open('testing.pkl', 'rb') as f:
     test = pickle.load(f)
        
with open('validation.pkl','rb') as f:
     val = pickle.load(f)
        
train_photos, train_labels = list(), list()
tp = list()
for file in train:
    if 'Cat' in file:
        output = 1.0
    else:
        output = 0.0
    photo = load_img(file)
    photo = img_to_array(photo)
    train_photos.append(photo)
    train_labels.append(output)
train_photos = asarray(train_photos)
train_labels = asarray(train_labels)

test_photos, test_labels = list(), list()
for file in test:
    if 'Cat' in file:
        output = 1.0
    else:
        output = 0.0
    photo = load_img(file)
    photo = img_to_array(photo)
    tp.append(photo)
    test_photos.append(photo)
    test_labels.append(output)
test_photos = asarray(test_photos)
test_labels = asarray(test_labels)

val_photos, val_labels = list(), list()
for file in val:
    if 'Cat' in file:
        output = 1.0
    else:
        output = 0.0
    photo = load_img(file)
    photo = img_to_array(photo)
    val_photos.append(photo)
    val_labels.append(output)
val_photos = asarray(val_photos)
val_labels = asarray(val_labels)

model= load_model("model.h5")


Using TensorFlow backend.
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarn

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [2]:
import pickle
from os import listdir
from numpy import asarray
from numpy import save
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
import numpy as np
from keras import layers
from keras.layers import Input,Dense,BatchNormalization,Flatten,Dropout,GlobalAveragePooling2D
from keras.models import Model, load_model
from keras.utils import layer_utils
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
import keras.backend as K
from keras.applications.vgg16 import VGG16
from keras.models import Model,load_model
from keras.optimizers import RMSprop

import optuna


N_TRAIN_EXAMPLES = len(train_photos)
N_VALID_EXAMPLES = len(val_photos)
BATCHSIZE = 1
CLASSES = 2
EPOCHS = 3
STUDY = None
N_TRIALS = 10
TAR_FILE = "basicnet_model.tar.gz"


def objective(trial):
    # Clear clutter from previous Keras session graphs.
#     clear_session()

    (x_train, y_train), (x_valid, y_valid) = (train_photos,train_labels),(val_photos,val_labels)
#     img_x, img_y = x_train.shape[1], x_train.shape[2]
#     x_train = x_train.reshape(-1, img_x, img_y, 1)[:N_TRAIN_EXAMPLES].astype("float32") / 255
#     x_valid = x_valid.reshape(-1, img_x, img_y, 1)[:N_VALID_EXAMPLES].astype("float32") / 255
#     y_train = y_train[:N_TRAIN_EXAMPLES]
#     y_valid = y_valid[:N_VALID_EXAMPLES]
#     input_shape = (img_x, img_y, 1)
    vgg16_model = VGG16(weights = 'imagenet', include_top = False)
    x = vgg16_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(CLASSES, activation = 'softmax')(x)
    model = Model(inputs = vgg16_model.input, outputs = predictions)
    
    for layer in vgg16_model.layers:
        layer.trainable = False

    # We compile our model with a sampled learning rate.
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    model.compile(
        loss="sparse_categorical_crossentropy", optimizer=RMSprop(lr=lr), metrics=["accuracy"]
    )

    model.fit(
        x_train,
        y_train,
        validation_data=(x_valid, y_valid),
        shuffle=True,
        batch_size=BATCHSIZE,
        epochs=EPOCHS,
        verbose=False,
    )

    # Evaluate the model accuracy on the validation set.
    score = model.evaluate(x_valid, y_valid, verbose=0)
    return score[1]

from keras.applications.vgg16 import VGG16
def create_classifier(trial):
    
    # We optimize the numbers of layers and their units.
    vgg16_model = VGG16(weights = 'imagenet', include_top = False)
    x = vgg16_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(CLASSES, activation = 'softmax')(x)
    model = Model(inputs = vgg16_model.input, outputs = predictions)
    
    for layer in vgg16_model.layers:
        layer.trainable = False
        
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    model.compile(
        loss="sparse_categorical_crossentropy", optimizer=RMSprop(lr=lr), metrics=["accuracy"]
    )

    model.fit(
        x_train,
        y_train,
        validation_data=(x_valid, y_valid),
        shuffle=True,
        batch_size=BATCHSIZE,
        epochs=EPOCHS,
        verbose=False,
    )

    # Evaluate the model accuracy on the validation set.
    score = model.evaluate(x_valid, y_valid, verbose=0)
    return score[1]


def objective(trial):
    classifier = create_classifier(trial)

    classifier.train(input_fn=train_input_fn, steps=TRAIN_STEPS)

    eval_results = classifier.evaluate(input_fn=eval_input_fn)

    return float(eval_results["accuracy"])


In [ ]:
study = optuna.study.create_study( direction='maximize')
study.optimize(objective, n_trials=1)

In [2]:
import tensorflow as tf
print(tf.__version__)

/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

1.13.1


In [ ]:
if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=10, timeout=600)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

#     print("  Value: {}".format(trial.value))

#     print("  Params: ")
#     for key, value in trial.params.items():
#         print("{}: {}".format(key, value))

In [ ]:
'Don\'t woryy'

In [ ]:
def get_data():

#     data_transforms  = transforms.Compose([ToTensorRescale()])
    train_dataset    = train_photos
    val_dataset      = val_photos
    train_labels = train_labels
    val_labels   = val_labels

    return train_dataset,val_dataset,train_labels,val_labels

def train(train_dataset, model, criterion, optimizer):

    model.train()
#     model.to(DEVICE)
    running_loss = 0

    for sample_batched in train_loader:        
        optimizer.zero_grad() 
        inputs, labels = sample_batched['image'].float(), sample_batched["label"]

        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)     
        log_ps = model(inputs)
        loss   = criterion(log_ps,labels)       
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    train_loss = running_loss/len(train_loader)

    return model, train_loss

def validate(val_loader, model, criterion):

    model.eval()
    model.to(DEVICE)
    accuracy = 0
    test_loss = 0

    with torch.no_grad():
        for sample_batched in val_loader:
            inputs,labels   = sample_batched["image"].float(), sample_batched["label"]
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            logs_ps = model(inputs)
            test_loss += criterion(logs_ps,labels)
            ps = torch.exp(logs_ps)
            top_ps, top_class = ps.topk(1,dim =1)
            equals = top_class == labels.view(*top_class.shape)
            accuracy += torch.mean(equals.type(torch.FloatTensor))
        test_loss_final = test_loss/len(val_loader)
        accuracy_final  = accuracy/len(val_loader)

    return model, test_loss_final, accuracy_final


In [ ]:
def build_model(trial):

    model = None
    params_optim = None

    if MODEL == "vgg16":
        model = PretrainedVGG16().float()
        params_optim = model.parameters()
    elif MODEL == "densenet121":
        model = PretrainedDenseNet121().float()
        params_optim = model.parameters()
    else:
        p = trial.suggest_float("dropout", 0.2, 0.5)
        model = BasicNet(p)
        params_optim = model.parameters()

    return model, params_optim

def objective(trial):
    global CHECKPOINTS_FILES_LIST
    #print("Performing trail {}".format(trial.number))

    # Generate the model.
    model, params_optim = build_model(trial)
    model.to(DEVICE)

    # Generate the optimizers.
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop"])
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    optimizer = getattr(optim, optimizer_name)(params_optim, lr=lr)

    # Get the cat/dog dataset.
    train_loader, valid_loader = get_dataloaders()
    criterion = nn.NLLLoss()
    losses_dict= {'train': {}, 'test': {}, 'accuracy': {}}
    start_epoch = 1
    accuracy = 0

    for epoch in range(start_epoch,EPOCHS+1):
        model, train_loss = train(train_loader, model, criterion, optimizer)
        model, test_loss, test_accuracy = validate(valid_loader, model, criterion)
        current_metrics = [epoch, train_loss, test_loss, test_accuracy]
        losses_dict["train"][epoch], losses_dict["test"][epoch] = train_loss, test_loss
        losses_dict["accuracy"][epoch] = test_accuracy
        accuracy += test_accuracy

        trial.report(test_accuracy, epoch)

        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

        if epoch % 5 == 0 :
            CHECKPOINTS_FILES_LIST= save_checkpoint(STUDY,model,optimizer, current_metrics,CHECKPOINTS_FILES_LIST,losses_dict, trial, str(MODEL))
#            plot_losses(losses_dict, epoch, trial.number)
    accuracy = accuracy/(EPOCHS+1) # TODO

    return accuracy

In [ ]:
def create_study(hpo_checkpoint_file):
    global STUDY
    load_checkpoint_flag = True
    
    if len(CHECKPOINTS_FILES_LIST) == 0:
        load_checkpoint_flag = False

    if load_checkpoint_flag:
        #print("Loading an existing study...")
        STUDY = joblib.load("hpo_study_checkpoint_" + MODEL + ".pkl")
        #check_status_last_trial()
        todo_trials = N_TRIALS - len(STUDY.trials_dataframe())
        if todo_trials > 0 :
            #print("There are {} trials to do out of {}".format(todo_trials, N_TRIALS))
            STUDY.optimize(objective, n_trials=todo_trials, timeout=600, callbacks=[hpo_monitor])
        else:
            pass
            #print("This study is finished. Nothing to do.")
    else:
        STUDY = optuna.create_study(direction = 'maximize', study_name = MODEL)
        STUDY.optimize(objective, n_trials=N_TRIALS, timeout=600, callbacks=[hpo_monitor])
